# MY CODE (NOT REQ)

## NOT REQ (Initial key is in form of bytes

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import csv


In [2]:
# Load the data from CSV
data = pd.read_csv('keystream_records_4lakh.csv')

# Separate the input features (8-byte key) and the targets (10 bytes of keystream)
X = data.iloc[:, :8].values  # First 8 columns are the 8-byte key
y = data.iloc[:, 8:18].values  # Next 10 columns are the 10 bytes of the keystream

# Convert the target to categorical (byte values range from 0 to 255)
y = y.astype(np.uint8)

# Split the data into training, validation, and testing sets (70-10-20 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)  # 0.67 * 0.3 ≈ 0.2


In [3]:
class A51KeystreamPredictor(nn.Module):
    def __init__(self):
        super(A51KeystreamPredictor, self).__init__()
        self.fc1 = nn.Linear(8, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 10)
        self.fc5 = nn.Linear(10, 10)
        self.fc6 = nn.Linear(10, 10)
        self.fc7 = nn.Linear(10, 10)
        self.output = nn.Linear(10, 256)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))
        x = torch.softmax(self.output(x), dim=1)
        return x


In [4]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test, byte_index):
    # Initialize the model, loss function, and optimizer
    model = A51KeystreamPredictor().to(device)
    criterion = nn.CrossEntropyLoss()  # Cross-Entropy Loss for classification
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Convert data to PyTorch tensors and move to GPU if available
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train[:, byte_index], dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val[:, byte_index], dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test[:, byte_index], dtype=torch.long).to(device)

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = torch.utils.data.TensorDataset(X_val_tensor, y_val_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

    # Early stopping parameters
    patience = 5
    best_val_loss = float('inf')
    best_model = None
    patience_counter = 0

    # Training the model
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct_train / total_train

        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

        val_loss /= len(val_loader)
        val_accuracy = correct_val / total_val

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                break

    # Load the best model
    model.load_state_dict(best_model)

    # Evaluate the model on the test set
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        test_accuracy = accuracy_score(y_test_tensor.cpu(), predicted.cpu())
        return test_accuracy

# Train and evaluate the model for each byte and print the accuracy
accuracies = []
for byte_index in range(10):
    print(f'\nTraining for byte {byte_index}...')
    accuracy = train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test, byte_index)
    accuracies.append(accuracy)
    print(f'Accuracy for byte {byte_index}: {accuracy * 100:.2f}%')

# Print all accuracies
print("\nAccuracies for all 10 bytes:")
for byte_index, accuracy in enumerate(accuracies):
    print(f'Byte {byte_index}: {accuracy * 100:.2f}%')


Using device: cuda

Training for byte 0...
Epoch 1/5, Loss: 5.5452, Accuracy: 0.0040, Val Loss: 5.5452, Val Accuracy: 0.0040
Epoch 2/5, Loss: 5.5452, Accuracy: 0.0040, Val Loss: 5.5452, Val Accuracy: 0.0042
Epoch 3/5, Loss: 5.5451, Accuracy: 0.0039, Val Loss: 5.5452, Val Accuracy: 0.0041
Epoch 4/5, Loss: 5.5451, Accuracy: 0.0040, Val Loss: 5.5452, Val Accuracy: 0.0040
Epoch 5/5, Loss: 5.5451, Accuracy: 0.0042, Val Loss: 5.5452, Val Accuracy: 0.0040
Accuracy for byte 0: 0.38%

Training for byte 1...
Epoch 1/5, Loss: 5.5452, Accuracy: 0.0040, Val Loss: 5.5452, Val Accuracy: 0.0038
Epoch 2/5, Loss: 5.5452, Accuracy: 0.0041, Val Loss: 5.5452, Val Accuracy: 0.0039
Epoch 3/5, Loss: 5.5451, Accuracy: 0.0041, Val Loss: 5.5452, Val Accuracy: 0.0039
Epoch 4/5, Loss: 5.5451, Accuracy: 0.0041, Val Loss: 5.5452, Val Accuracy: 0.0039
Epoch 5/5, Loss: 5.5451, Accuracy: 0.0043, Val Loss: 5.5452, Val Accuracy: 0.0045
Accuracy for byte 1: 0.39%

Training for byte 2...
Epoch 1/5, Loss: 5.5452, Accuracy: 

## VERY FIRST CODE WITHOUT OVERFIT CHK

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np


In [6]:
# Load the data from CSV
data = pd.read_csv('keystream_records.csv')

# Separate the input features (64-bit key) and the targets (10 bytes of keystream)
X = data.iloc[:, :64].values  # First 64 columns are the 64-bit key
y = data.iloc[:, 64:74].values   # Next 10 columns are the 10 bytes of the keystream

# Convert the target to categorical (byte values range from 0 to 255)
y = y.astype(np.uint8)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
class KeystreamClassifier(nn.Module):
    def __init__(self):
        super(KeystreamClassifier, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 256)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [8]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate(X_train, y_train, X_test, y_test, byte_index):
    # Initialize the model, loss function, and optimizer
    model = KeystreamClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Convert data to PyTorch tensors and move to GPU if available
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train[:, byte_index], dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test[:, byte_index], dtype=torch.long).to(device)
    
    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    
    # Training the model
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu(), predicted.cpu())
        return accuracy

# Train and evaluate the model for each byte and print the accuracy
accuracies = []
for byte_index in range(10):
    print(f'\nTraining for byte {byte_index}...')
    accuracy = train_and_evaluate(X_train, y_train, X_test, y_test, byte_index)
    accuracies.append(accuracy)
    print(f'Accuracy for byte {byte_index}: {accuracy * 100:.2f}%')

# Print all accuracies
print("\nAccuracies for all 10 bytes:")
for byte_index, accuracy in enumerate(accuracies):
    print(f'Byte {byte_index}: {accuracy * 100:.2f}%')


Using device: cuda

Training for byte 0...
Epoch 1/20, Loss: 5.546666142654419
Epoch 2/20, Loss: 5.543869366455078
Epoch 3/20, Loss: 5.536435187530517
Epoch 4/20, Loss: 5.514980736160278
Epoch 5/20, Loss: 5.475875542449951
Epoch 6/20, Loss: 5.421879569625855
Epoch 7/20, Loss: 5.360091432571411
Epoch 8/20, Loss: 5.294062951278686
Epoch 9/20, Loss: 5.228262239456177
Epoch 10/20, Loss: 5.1649985584259035
Epoch 11/20, Loss: 5.107959839630127
Epoch 12/20, Loss: 5.055957280731201
Epoch 13/20, Loss: 5.011220754241943
Epoch 14/20, Loss: 4.97015382232666
Epoch 15/20, Loss: 4.932823988723755
Epoch 16/20, Loss: 4.899966220855713
Epoch 17/20, Loss: 4.8698786418914795
Epoch 18/20, Loss: 4.842659124374389
Epoch 19/20, Loss: 4.818942027664185
Epoch 20/20, Loss: 4.7959378032684326
Accuracy for byte 0: 0.38%

Training for byte 1...
Epoch 1/20, Loss: 5.546703688430786
Epoch 2/20, Loss: 5.544201615905762
Epoch 3/20, Loss: 5.538968728637696
Epoch 4/20, Loss: 5.521449309921264
Epoch 5/20, Loss: 5.487367841

In [ ]:
# Load the data from CSV
data = pd.read_csv('keystream_records_10lakh.csv')

# Separate the input features (64-bit key) and the targets (10 bytes of keystream)
X = data.iloc[:, :64].values  # First 64 columns are the 64-bit key
y = data.iloc[:, 64:74].values   # Next 10 columns are the 10 bytes of the keystream

# Convert the target to categorical (byte values range from 0 to 255)
y = y.astype(np.uint8)

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate(X_train, y_train, X_test, y_test, byte_index):
    # Initialize the model, loss function, and optimizer
    model = KeystreamClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Convert data to PyTorch tensors and move to GPU if available
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train[:, byte_index], dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test[:, byte_index], dtype=torch.long).to(device)
    
    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    
    # Training the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(y_test_tensor.cpu(), predicted.cpu())
        return accuracy

# Train and evaluate the model for each byte and print the accuracy
accuracies = []
for byte_index in range(10):
    print(f'\nTraining for byte {byte_index}...')
    accuracy = train_and_evaluate(X_train, y_train, X_test, y_test, byte_index)
    accuracies.append(accuracy)
    print(f'Accuracy for byte {byte_index}: {accuracy * 100:.2f}%')

# Print all accuracies
print("\nAccuracies for all 10 bytes:")
for byte_index, accuracy in enumerate(accuracies):
    print(f'Byte {byte_index}: {accuracy * 100:.2f}%')


# TISHIKA

## NOT WORKING

In [5]:
# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate(X_train, y_train, X_test, y_test, byte_index):
    # Initialize the model
    model = build_model()

    # Convert the labels to categorical
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)  # 256 possible byte values
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Transfer data to GPU
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    
    # Train the model
    model.fit(X_train_gpu, y_train_gpu, epochs=1, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

# Train and evaluate the model for each byte and print the accuracy
accuracies = []
for byte_index in range(10):  # 10 bytes
    print(f'\nTraining for byte {byte_index}...')
    accuracy = train_and_evaluate(X_train, y_train, X_test, y_test, byte_index)
    accuracies.append(accuracy)
    print(f'Accuracy for byte {byte_index}: {accuracy * 100:.2f}%')


Training for byte 0...
33750/33750 [==============================] - 256s 7ms/step - loss: 5.5776 - accuracy: 0.0039 - val_loss: 5.5535 - val_accuracy: 0.0041
Accuracy for byte 0: 0.39%

Training for byte 1...


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Print all accuracies
print("Accuracies for all 10 bytes:")
for byte_index, accuracy in enumerate(accuracies):
    print(f'Byte {byte_index}: {accuracy * 100:.2f}%')

## WORKING 10 LAKH

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


print(tf.__version__)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load the data from CSV
data = pd.read_csv('keystream_records_10lakh.csv')

# Separate the input features (64-bit key) and the targets (10 bytes of keystream)
X = data.iloc[:, :64].values  # First 64 columns are the 64-bit key
y = data.iloc[:, 64:74].values  # Next 10 columns are the 10 bytes of the keystream

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X to be (num_samples, 64, 1) for Conv1D
X_train_reshaped = X_train.reshape(-1, 64, 1)
X_test_reshaped = X_test.reshape(-1, 64, 1)

# Function to build the model
def build_model():
    input_layer = Input(shape=(64, 1))
    x = Conv1D(32, 4, activation='relu', padding='same', strides=2)(input_layer)
    x = BatchNormalization()(x)
    x = Conv1D(64, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(128, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(256, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    output_layer = Dense(256, activation='softmax')(x)  # 256 units for each byte
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate_byte_0(X_train, y_train, X_test, y_test):
    byte_index = 0
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_1(X_train, y_train, X_test, y_test):
    byte_index = 1
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

# Repeat similar function definitions for bytes 2 through 9

def train_and_evaluate_byte_2(X_train, y_train, X_test, y_test):
    byte_index = 2
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_3(X_train, y_train, X_test, y_test):
    byte_index = 3
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_4(X_train, y_train, X_test, y_test):
    byte_index = 4
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_5(X_train, y_train, X_test, y_test):
    byte_index = 5
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_6(X_train, y_train, X_test, y_test):
    byte_index = 6
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_7(X_train, y_train, X_test, y_test):
    byte_index = 7
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_8(X_train, y_train, X_test, y_test):
    byte_index = 8
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_9(X_train, y_train, X_test, y_test):
    byte_index = 9
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy



Num GPUs Available:  1
2.10.1


In [3]:
# Train and evaluate the model for each byte and print the accuracy
#accuracies = []

print('\nTraining for byte 0...')
accuracy = train_and_evaluate_byte_0(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 0: {accuracy * 100:.2f}%')


Training for byte 0...
Epoch 1/50
33750/33750 [==============================] - 245s 7ms/step - loss: 5.5779 - accuracy: 0.0039 - val_loss: 5.5521 - val_accuracy: 0.0042
Epoch 2/50
33750/33750 [==============================] - 245s 7ms/step - loss: 5.5531 - accuracy: 0.0039 - val_loss: 5.5534 - val_accuracy: 0.0036
Epoch 3/50
33750/33750 [==============================] - 238s 7ms/step - loss: 5.5541 - accuracy: 0.0038 - val_loss: 5.5548 - val_accuracy: 0.0042
Epoch 4/50
33750/33750 [==============================] - 243s 7ms/step - loss: 5.5543 - accuracy: 0.0040 - val_loss: 5.5542 - val_accuracy: 0.0039
Epoch 5/50
33750/33750 [==============================] - 242s 7ms/step - loss: 5.5541 - accuracy: 0.0041 - val_loss: 5.5556 - val_accuracy: 0.0041
Epoch 6/50
33750/33750 [==============================] - 243s 7ms/step - loss: 5.5535 - accuracy: 0.0042 - val_loss: 5.5543 - val_accuracy: 0.0039
Accuracy for byte 0: 0.40%


In [2]:
print('\nTraining for byte 1...')
accuracy = train_and_evaluate_byte_1(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 1: {accuracy * 100:.2f}%')


Training for byte 1...
Epoch 1/50
33750/33750 [==============================] - 226s 7ms/step - loss: 5.5779 - accuracy: 0.0038 - val_loss: 5.5508 - val_accuracy: 0.0040
Epoch 2/50
33750/33750 [==============================] - 227s 7ms/step - loss: 5.5530 - accuracy: 0.0039 - val_loss: 5.5547 - val_accuracy: 0.0039
Epoch 3/50
33750/33750 [==============================] - 228s 7ms/step - loss: 5.5541 - accuracy: 0.0038 - val_loss: 5.5550 - val_accuracy: 0.0036
Epoch 4/50
33750/33750 [==============================] - 225s 7ms/step - loss: 5.5542 - accuracy: 0.0040 - val_loss: 5.5560 - val_accuracy: 0.0041
Epoch 5/50
33750/33750 [==============================] - 262s 8ms/step - loss: 5.5540 - accuracy: 0.0040 - val_loss: 5.5572 - val_accuracy: 0.0039
Epoch 6/50
33750/33750 [==============================] - 259s 8ms/step - loss: 5.5533 - accuracy: 0.0043 - val_loss: 5.5588 - val_accuracy: 0.0041
Accuracy for byte 1: 0.40%


In [2]:
print('\nTraining for byte 2...')
accuracy = train_and_evaluate_byte_2(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 2: {accuracy * 100:.2f}%')


Training for byte 2...
Epoch 1/50
33750/33750 [==============================] - 292s 9ms/step - loss: 5.5775 - accuracy: 0.0040 - val_loss: 5.5529 - val_accuracy: 0.0036
Epoch 2/50
33750/33750 [==============================] - 309s 9ms/step - loss: 5.5531 - accuracy: 0.0039 - val_loss: 5.5528 - val_accuracy: 0.0037
Epoch 3/50
33750/33750 [==============================] - 328s 10ms/step - loss: 5.5539 - accuracy: 0.0040 - val_loss: 5.5574 - val_accuracy: 0.0039
Epoch 4/50
33750/33750 [==============================] - 285s 8ms/step - loss: 5.5542 - accuracy: 0.0040 - val_loss: 5.5564 - val_accuracy: 0.0040
Epoch 5/50
33750/33750 [==============================] - 687s 20ms/step - loss: 5.5539 - accuracy: 0.0041 - val_loss: 5.5571 - val_accuracy: 0.0038
Epoch 6/50
33750/33750 [==============================] - 239s 7ms/step - loss: 5.5530 - accuracy: 0.0043 - val_loss: 5.5565 - val_accuracy: 0.0042
Epoch 7/50
33750/33750 [==============================] - 233s 7ms/step - loss: 5.5519

NameError: name 'accuracies' is not defined

In [3]:
print(f'Accuracy for byte 2: {accuracy * 100:.2f}%')

Accuracy for byte 2: 0.40%


In [2]:
print('\nTraining for byte 3...')
accuracy = train_and_evaluate_byte_3(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 3: {accuracy * 100:.2f}%')


Training for byte 3...
Epoch 1/50
33750/33750 [==============================] - 279s 8ms/step - loss: 5.5777 - accuracy: 0.0039 - val_loss: 5.5529 - val_accuracy: 0.0038
Epoch 2/50
33750/33750 [==============================] - 263s 8ms/step - loss: 5.5530 - accuracy: 0.0039 - val_loss: 5.5551 - val_accuracy: 0.0038
Epoch 3/50
33750/33750 [==============================] - 272s 8ms/step - loss: 5.5540 - accuracy: 0.0040 - val_loss: 5.5536 - val_accuracy: 0.0041
Epoch 4/50
33750/33750 [==============================] - 263s 8ms/step - loss: 5.5542 - accuracy: 0.0041 - val_loss: 5.5555 - val_accuracy: 0.0039
Epoch 5/50
33750/33750 [==============================] - 272s 8ms/step - loss: 5.5537 - accuracy: 0.0041 - val_loss: 5.5577 - val_accuracy: 0.0038
Epoch 6/50
33750/33750 [==============================] - 265s 8ms/step - loss: 5.5528 - accuracy: 0.0043 - val_loss: 5.5574 - val_accuracy: 0.0040
Accuracy for byte 3: 0.38%


In [2]:
print('\nTraining for byte 4...')
accuracy = train_and_evaluate_byte_4(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 4: {accuracy * 100:.2f}%')


Training for byte 4...
Epoch 1/50
33750/33750 [==============================] - 301s 7ms/step - loss: 5.5779 - accuracy: 0.0039 - val_loss: 5.5511 - val_accuracy: 0.0040
Epoch 2/50
33750/33750 [==============================] - 239s 7ms/step - loss: 5.5530 - accuracy: 0.0038 - val_loss: 5.5548 - val_accuracy: 0.0040
Epoch 3/50
33750/33750 [==============================] - 239s 7ms/step - loss: 5.5540 - accuracy: 0.0040 - val_loss: 5.5550 - val_accuracy: 0.0039
Epoch 4/50
33750/33750 [==============================] - 238s 7ms/step - loss: 5.5541 - accuracy: 0.0041 - val_loss: 5.5562 - val_accuracy: 0.0040
Epoch 5/50
33750/33750 [==============================] - 228s 7ms/step - loss: 5.5534 - accuracy: 0.0041 - val_loss: 5.5593 - val_accuracy: 0.0036
Epoch 6/50
33750/33750 [==============================] - 303s 9ms/step - loss: 5.5523 - accuracy: 0.0044 - val_loss: 5.5590 - val_accuracy: 0.0039
Accuracy for byte 4: 0.38%


In [2]:
print('\nTraining for byte 5...')
accuracy = train_and_evaluate_byte_5(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 5: {accuracy * 100:.2f}%')


Training for byte 5...
Epoch 1/50
33750/33750 [==============================] - 263s 8ms/step - loss: 5.5777 - accuracy: 0.0040 - val_loss: 5.5531 - val_accuracy: 0.0038
Epoch 2/50
33750/33750 [==============================] - 246s 7ms/step - loss: 5.5530 - accuracy: 0.0039 - val_loss: 5.5517 - val_accuracy: 0.0037
Epoch 3/50
33750/33750 [==============================] - 238s 7ms/step - loss: 5.5538 - accuracy: 0.0039 - val_loss: 5.5546 - val_accuracy: 0.0040
Epoch 4/50
33750/33750 [==============================] - 250s 7ms/step - loss: 5.5543 - accuracy: 0.0039 - val_loss: 5.5555 - val_accuracy: 0.0041
Epoch 5/50
33750/33750 [==============================] - 256s 8ms/step - loss: 5.5540 - accuracy: 0.0042 - val_loss: 5.5564 - val_accuracy: 0.0037
Epoch 6/50
33750/33750 [==============================] - 264s 8ms/step - loss: 5.5534 - accuracy: 0.0041 - val_loss: 5.5564 - val_accuracy: 0.0039
Epoch 7/50
33750/33750 [==============================] - 253s 7ms/step - loss: 5.5523 -

In [2]:
print('\nTraining for byte 6...')
accuracy = train_and_evaluate_byte_6(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 6: {accuracy * 100:.2f}%')


Training for byte 6...
Epoch 1/50
33750/33750 [==============================] - 244s 7ms/step - loss: 5.5776 - accuracy: 0.0039 - val_loss: 5.5521 - val_accuracy: 0.0037
Epoch 2/50
33750/33750 [==============================] - 250s 7ms/step - loss: 5.5529 - accuracy: 0.0039 - val_loss: 5.5561 - val_accuracy: 0.0038
Epoch 3/50
33750/33750 [==============================] - 256s 8ms/step - loss: 5.5541 - accuracy: 0.0039 - val_loss: 5.5557 - val_accuracy: 0.0038
Epoch 4/50
33750/33750 [==============================] - 248s 7ms/step - loss: 5.5542 - accuracy: 0.0040 - val_loss: 5.5548 - val_accuracy: 0.0043
Epoch 5/50
33750/33750 [==============================] - 264s 8ms/step - loss: 5.5541 - accuracy: 0.0041 - val_loss: 5.5569 - val_accuracy: 0.0041
Epoch 6/50
33750/33750 [==============================] - 270s 8ms/step - loss: 5.5534 - accuracy: 0.0043 - val_loss: 5.5570 - val_accuracy: 0.0040
Accuracy for byte 6: 0.39%


In [2]:
print('\nTraining for byte 7...')
accuracy = train_and_evaluate_byte_7(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 7: {accuracy * 100:.2f}%')


Training for byte 7...
Epoch 1/50
33750/33750 [==============================] - 229s 7ms/step - loss: 5.5780 - accuracy: 0.0038 - val_loss: 5.5528 - val_accuracy: 0.0038
Epoch 2/50
33750/33750 [==============================] - 237s 7ms/step - loss: 5.5531 - accuracy: 0.0040 - val_loss: 5.5526 - val_accuracy: 0.0042
Epoch 3/50
33750/33750 [==============================] - 236s 7ms/step - loss: 5.5540 - accuracy: 0.0039 - val_loss: 5.5531 - val_accuracy: 0.0038
Epoch 4/50
33750/33750 [==============================] - 244s 7ms/step - loss: 5.5542 - accuracy: 0.0039 - val_loss: 5.5557 - val_accuracy: 0.0036
Epoch 5/50
33750/33750 [==============================] - 332s 10ms/step - loss: 5.5540 - accuracy: 0.0043 - val_loss: 5.5551 - val_accuracy: 0.0041
Epoch 6/50
33750/33750 [==============================] - 372s 11ms/step - loss: 5.5531 - accuracy: 0.0041 - val_loss: 5.5584 - val_accuracy: 0.0039
Epoch 7/50
33750/33750 [==============================] - 373s 11ms/step - loss: 5.551

In [2]:
print('\nTraining for byte 8...')
accuracy = train_and_evaluate_byte_8(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 8: {accuracy * 100:.2f}%')


Training for byte 8...
Epoch 1/50
33750/33750 [==============================] - 237s 7ms/step - loss: 5.5777 - accuracy: 0.0039 - val_loss: 5.5511 - val_accuracy: 0.0040
Epoch 2/50
33750/33750 [==============================] - 229s 7ms/step - loss: 5.5530 - accuracy: 0.0039 - val_loss: 5.5551 - val_accuracy: 0.0037
Epoch 3/50
33750/33750 [==============================] - 234s 7ms/step - loss: 5.5541 - accuracy: 0.0040 - val_loss: 5.5539 - val_accuracy: 0.0036
Epoch 4/50
33750/33750 [==============================] - 232s 7ms/step - loss: 5.5543 - accuracy: 0.0041 - val_loss: 5.5541 - val_accuracy: 0.0039
Epoch 5/50
33750/33750 [==============================] - 229s 7ms/step - loss: 5.5540 - accuracy: 0.0042 - val_loss: 5.5581 - val_accuracy: 0.0038
Epoch 6/50
33750/33750 [==============================] - 223s 7ms/step - loss: 5.5533 - accuracy: 0.0042 - val_loss: 5.5579 - val_accuracy: 0.0039
Accuracy for byte 8: 0.40%


In [2]:
print('\nTraining for byte 9...')
accuracy = train_and_evaluate_byte_9(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 9: {accuracy * 100:.2f}%')


Training for byte 9...
Epoch 1/50
33750/33750 [==============================] - 226s 7ms/step - loss: 5.5779 - accuracy: 0.0039 - val_loss: 5.5525 - val_accuracy: 0.0039
Epoch 2/50
33750/33750 [==============================] - 218s 6ms/step - loss: 5.5530 - accuracy: 0.0039 - val_loss: 5.5537 - val_accuracy: 0.0038
Epoch 3/50
33750/33750 [==============================] - 221s 7ms/step - loss: 5.5539 - accuracy: 0.0040 - val_loss: 5.5549 - val_accuracy: 0.0037
Epoch 4/50
33750/33750 [==============================] - 221s 7ms/step - loss: 5.5542 - accuracy: 0.0039 - val_loss: 5.5546 - val_accuracy: 0.0038
Epoch 5/50
33750/33750 [==============================] - 221s 7ms/step - loss: 5.5537 - accuracy: 0.0042 - val_loss: 5.5588 - val_accuracy: 0.0035
Epoch 6/50
33750/33750 [==============================] - 225s 7ms/step - loss: 5.5528 - accuracy: 0.0042 - val_loss: 5.5577 - val_accuracy: 0.0038
Accuracy for byte 9: 0.40%
